In [1]:
!pip install unidecode
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.6 MB/s eta 0:00:00


In [2]:
import torch
import os
import pandas as pd

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

NUM_ITERS = 80000
BATCH_SIZE = 4
PRINT_PER_ITER = 1
VALID_PER_ITER = 2000
MAX_SAMPLE_VALID = 10000

MAX_LR = 0.0003    # lr will inscrease from 2e-5 to MAX_LR in iter 0 -> iter NUM_ITERS * PCT_START, then decrease to 2e-5
PCT_START = 0.1

ROOT_PATH = '/kaggle/working/'
CHECKPOINT = os.path.join(ROOT_PATH,'checkpoints')
EXPORT = os.path.join(ROOT_PATH,'weights')

MAXLEN = 46
NGRAM = 5
alphabets = 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~ ]'
PERCENT_NOISE = 0.3 # 0 - 2 word in 5-grams


In [4]:
os.makedirs(ROOT_PATH, exist_ok=True)
os.makedirs(CHECKPOINT, exist_ok=True)
os.makedirs(EXPORT, exist_ok=True)

In [5]:
from datasets import load_dataset
dataset = load_dataset("tiendoan/ngrams_vnc_dataset")

Generating train split:   0%|          | 0/6579290 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/35999 [00:00<?, ? examples/s]

In [6]:
val_size = int(len(dataset['train'])* 0.15)
train_dataset = dataset['train'][:-val_size]
val_dataset = dataset['train'][-val_size:]
test_dataset = dataset['test']

In [7]:
class Vocab():
    def __init__(self, chars):
        self.pad = 0
        self.go = 1
        self.eos = 2

        self.chars = chars
        self.c2i = {c: i + 3 for i, c in enumerate(chars)}
        self.i2c = {i + 3: c for i, c in enumerate(chars)}

        self.i2c[0] = '<pad>'
        self.i2c[1] = '<sos>'
        self.i2c[2] = '<eos>'

    def encode(self, chars):
        return [self.go] + [self.c2i[c] for c in chars] + [self.eos]

    def decode(self, ids):
        first = 1 if self.go in ids else 0
        last = ids.index(self.eos) if self.eos in ids else None
        return ''.join([self.i2c[i] for i in ids[first:last]])

    def __len__(self):
        return len(self.c2i) + 3

    def batch_decode(self, arr):
        return [self.decode(ids) for ids in arr]

    def __str__(self):
        return self.chars

In [8]:
import torch
import numpy as np

class AutoCorrectDataset(torch.utils.data.Dataset):
    def __init__(self, ngrams, noise_ngrams, vocab, maxlen):
        self.ngrams = ngrams
        self.noise_ngrams = noise_ngrams
        self.vocab = vocab
        self.maxlen = maxlen

    def __getitem__(self, idx):
        correct_sent = self.ngrams[idx]
        noise_sent = self.noise_ngrams[idx]

        correct_sent_idxs = self.vocab.encode(correct_sent)
        noise_sent_idxs = self.vocab.encode(noise_sent)

        src_len = len(noise_sent_idxs)
        if self.maxlen - src_len < 0:
            noise_sent_idxs = noise_sent_idxs[:self.maxlen]
            src_len = len(noise_sent_idxs)
            print("Over length in src")
        src = np.concatenate((
            noise_sent_idxs,
            np.zeros(self.maxlen - src_len, dtype=np.int32)))

        tgt_len = len(correct_sent_idxs)
        if self.maxlen - tgt_len < 0:
            correct_sent_idxs = correct_sent_idxs[:self.maxlen]
            tgt_len = len(correct_sent_idxs)
            print("Over length in target")
        tgt = np.concatenate((
            correct_sent_idxs,
            np.zeros(self.maxlen - tgt_len, dtype=np.int32)))

        return {
            'src': torch.LongTensor(src),
            'tgt': torch.LongTensor(tgt),
        }

    def __len__(self):
        return len(self.ngrams)



In [9]:
import torch
from torch import nn


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, padding_idx, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim
        self.padding_idx = padding_idx

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 2))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            true_dist[:, self.padding_idx] = 0
            mask = torch.nonzero(target.data == self.padding_idx, as_tuple=False)
            if mask.dim() > 0:
                true_dist.index_fill_(0, mask.squeeze(), 0.0)

        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        """
        src: src_len x batch_size
        outputs: src_len x batch_size x hid_dim
        hidden: batch_size x hid_dim
        """

        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))

        return outputs, hidden


class Attention(nn.Module):

    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: batch_size x hid_dim
        encoder_outputs: src_len x batch_size x hid_dim,
        outputs: batch_size x src_len
        """

        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        attention = self.v(energy).squeeze(2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        """
        inputs: batch_size
        hidden: batch_size x hid_dim
        encoder_outputs: src_len x batch_size x hid_dim
        """

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        a = self.attention(hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted = torch.bmm(a, encoder_outputs)

        weighted = weighted.permute(1, 0, 2)

        rnn_input = torch.cat((embedded, weighted), dim=2)

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))

        return prediction, hidden.squeeze(0), a.squeeze(1)


class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, encoder_embbeded, decoder_embedded, encoder_hidden, decoder_hidden,
                 encoder_dropout=0.1, decoder_dropout=0.1):
        super().__init__()

        attn = Attention(encoder_hidden, decoder_hidden)

        self.encoder = Encoder(input_dim, encoder_embbeded, encoder_hidden, decoder_hidden, encoder_dropout)
        self.decoder = Decoder(output_dim, decoder_embedded, encoder_hidden, decoder_hidden, decoder_dropout, attn)

    def forward_encoder(self, src):
        """
        src: timestep x batch_size x channel
        hidden: batch_size x hid_dim
        encoder_outputs: src_len x batch_size x hid_dim
        """

        encoder_outputs, hidden = self.encoder(src)

        return (hidden, encoder_outputs)

    def forward_decoder(self, tgt, memory):
        """
        tgt: timestep x batch_size
        hidden: batch_size x hid_dim
        encouder: src_len x batch_size x hid_dim
        output: batch_size x 1 x vocab_size
        """
        tgt = tgt[-1]
        hidden, encoder_outputs = memory
        output, hidden, _ = self.decoder(tgt, hidden, encoder_outputs)
        output = output.unsqueeze(1)

        return output, (hidden, encoder_outputs)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        src: time_step x batch_size
        trg: time_step x batch_size
        outputs: batch_size x time_step x vocab_size
        """
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        device = src.device

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        encoder_outputs, hidden = self.encoder(src)

        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            top1 = output.argmax(1)

            # Teacher force
            rnd_teacher = torch.rand(1).item()
            teacher_force = rnd_teacher < teacher_forcing_ratio
            input = trg[t, :] if teacher_force else top1
      

        outputs = outputs.transpose(0, 1).contiguous()
        return outputs

    def expand_memory(self, memory, beam_size):
        hidden, encoder_outputs = memory
        hidden = hidden.repeat(beam_size, 1)
        encoder_outputs = encoder_outputs.repeat(1, beam_size, 1)

        return (hidden, encoder_outputs)

    def get_memory(self, memory, i):
        hidden, encoder_outputs = memory
        hidden = hidden[[i]]
        encoder_outputs = encoder_outputs[:, [i], :]

        return (hidden, encoder_outputs)

In [11]:
from torch.nn.functional import log_softmax, softmax

def translate(src, model, max_seq_length=128, sos_token=1, eos_token=2):
    "data: Bxsrc_len"
    model.eval()
    device = src.device

    with torch.no_grad():
        # src = model.cnn(img)
        src = src.transpose(1, 0)  # src_len x B
        memory = model.forward_encoder(src)

        translated_sentence = [[sos_token] * src.shape[1]]
        char_probs = [[1] * src.shape[1]]

        max_length = 0

        while max_length <= max_seq_length and not all(np.any(np.asarray(translated_sentence).T == eos_token, axis=1)):
            tgt_inp = torch.LongTensor(translated_sentence).to(device)

            #            output = model(img, tgt_inp, tgt_key_padding_mask=None)
            #            output = model.transformer(src, tgt_inp, tgt_key_padding_mask=None)
            output, memory = model.forward_decoder(tgt_inp, memory)
            output = softmax(output, dim=-1)
            output = output.to('cpu')

            values, indices = torch.topk(output, 5)

            indices = indices[:, -1, 0]
            indices = indices.tolist()

            values = values[:, -1, 0]
            values = values.tolist()
            char_probs.append(values)

            translated_sentence.append(indices)
            max_length += 1

            del output

        translated_sentence = np.asarray(translated_sentence).T

        char_probs = np.asarray(char_probs).T
        char_probs = np.multiply(char_probs, translated_sentence > 3)
        char_probs = np.sum(char_probs, axis=-1) / (char_probs > 0).sum(-1)

    return translated_sentence, char_probs

In [12]:
from torch.optim import Adam, SGD, AdamW
import torch
from torch.optim.lr_scheduler import OneCycleLR
import numpy as np
import os
import time
from torch.utils.data import DataLoader
from tqdm import tqdm

class Trainer():
    def __init__(self, alphabets_, train_dataset,val_dataset):

        self.vocab = Vocab(alphabets_)
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        print("Total training samples: ", len(self.train_dataset['input']))
        print("Total valid samples: ", len(self.val_dataset['input']))

        INPUT_DIM = self.vocab.__len__()
        OUTPUT_DIM = self.vocab.__len__()

        self.device = DEVICE
        self.num_iters = NUM_ITERS

        self.batch_size = BATCH_SIZE
        self.print_every = PRINT_PER_ITER
        self.valid_every = VALID_PER_ITER

        self.checkpoint = os.path.join(CHECKPOINT,'checkpoint.pth')
        self.export_weights = os.path.join(EXPORT,'model.pth')
        self.metrics = MAX_SAMPLE_VALID


        self.iter = 0

        self.model = Seq2Seq(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM, encoder_embbeded=ENC_EMB_DIM,
                             decoder_embedded=DEC_EMB_DIM,
                             encoder_hidden=ENC_HID_DIM, decoder_hidden=DEC_HID_DIM, encoder_dropout=ENC_DROPOUT,
                             decoder_dropout=DEC_DROPOUT)

        self.optimizer = AdamW(self.model.parameters(), betas=(0.9, 0.98), eps=1e-09)
        self.scheduler = OneCycleLR(self.optimizer, total_steps=self.num_iters, pct_start=PCT_START, max_lr=MAX_LR)

        self.criterion = LabelSmoothingLoss(len(self.vocab), padding_idx=self.vocab.pad, smoothing=0.1)

        self.train_gen = self.data_gen(self.train_dataset, self.vocab, is_train=True)
        self.valid_gen = self.data_gen(self.val_dataset, self.vocab, is_train=False)

        self.train_losses = []

        # to device
        self.model.to(self.device)
        self.criterion.to(self.device)

    def data_gen(self, dataset, vocab, is_train=True):
        dataset = AutoCorrectDataset(ngrams = dataset['output'],noise_ngrams=dataset['input'], vocab=vocab, maxlen=MAXLEN)

        shuffle = bool(is_train)
        return DataLoader(
            dataset, batch_size=BATCH_SIZE, shuffle=shuffle, drop_last=False
        )

    def step(self, batch):
        self.model.train()

        batch = self.batch_to_device(batch)
        src, tgt = batch['src'], batch['tgt']
        src, tgt = src.transpose(1, 0), tgt.transpose(1, 0)  # batch x src_len -> src_len x batch

        outputs = self.model(src, tgt)  # src : src_len x B, outpus : B x tgt_len x vocab

        #        loss = self.criterion(rearrange(outputs, 'b t v -> (b t) v'), rearrange(tgt_output, 'b o -> (b o)'))
        outputs = outputs.view(-1, outputs.size(2))  # flatten(0, 1)

        tgt_output = tgt.transpose(0, 1).reshape(-1)  # flatten()   # tgt: tgt_len xB , need convert to B x tgt_len

        loss = self.criterion(outputs, tgt_output)

        self.optimizer.zero_grad()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1)

        self.optimizer.step()
        self.scheduler.step()

        return loss.item()

    def train(self):
        print("Begin training from iter: ", self.iter)
        total_loss = 0

        total_loader_time = 0
        total_gpu_time = 0
        best_cer = -1

        data_iter = iter(self.train_gen)
        for _ in range(self.num_iters):
            self.iter += 1

            start = time.time()

            try:
                batch = next(data_iter)
            except StopIteration:
                data_iter = iter(self.train_gen)
                batch = next(data_iter)

            total_loader_time += time.time() - start

            start = time.time()
            loss = self.step(batch)
            total_gpu_time += time.time() - start

            total_loss += loss
            self.train_losses.append((self.iter, loss))

            if self.iter % self.print_every == 0:
                info = 'iter: {:06d} - train loss: {:.3f} - lr: {:.2e} - load time: {:.2f} - gpu time: {:.2f}'.format(
                    self.iter,
                    total_loss / self.print_every, self.optimizer.param_groups[0]['lr'],
                    total_loader_time, total_gpu_time)

                total_loss = 0
                total_loader_time = 0
                total_gpu_time = 0
                print(info)


            if self.iter % self.valid_every == 0:
                val_loss, preds, actuals, inp_sents = self.validate()
                cer, wer = self.precision(self.metrics)

                info = 'iter: {:06d} - valid loss: {:.3f} - cer: {:.3f} '.format(
                    self.iter, val_loss, cer)
                print(info)
                print("--- Sentence predict ---")
                for pred, inp, label in zip(preds, inp_sents, actuals):
                    infor_predict = f'Pred: {pred} - Inp: {inp} - Label: {label}'
                    print(infor_predict)

                if cer > best_cer:
                    self.save_weights(self.export_weights)
                    best_cer = cer
                self.save_checkpoint(self.checkpoint)

    def validate(self):
        self.model.eval()

        total_loss = []
        max_step = self.metrics / self.batch_size
        with torch.no_grad():
            for step, batch in enumerate(self.valid_gen):
                batch = self.batch_to_device(batch)
                src, tgt = batch['src'], batch['tgt']
                src, tgt = src.transpose(1, 0), tgt.transpose(1, 0)

                outputs = self.model(src, tgt, 0)  # turn off teaching force

                outputs = outputs.flatten(0, 1)
                tgt_output = tgt.flatten()
                loss = self.criterion(outputs, tgt_output)

                total_loss.append(loss.item())

                preds, actuals, inp_sents, probs = self.predict(5)

                del outputs
                del loss
                if step > max_step:
                    break

        total_loss = np.mean(total_loss)
        self.model.train()

        return total_loss, preds[:3], actuals[:3], inp_sents[:3]

    def predict(self, sample=None):
        pred_sents = []
        actual_sents = []
        inp_sents = []

        for batch in self.valid_gen:
            batch = self.batch_to_device(batch)

            translated_sentence, prob = translate(batch['src'], self.model)
            

            pred_sent = self.vocab.batch_decode(translated_sentence.tolist())
            actual_sent = self.vocab.batch_decode(batch['tgt'].tolist())
            inp_sent = self.vocab.batch_decode(batch['src'].tolist())

            pred_sents.extend(pred_sent)
            actual_sents.extend(actual_sent)
            inp_sents.extend(inp_sent)

            if sample is not None and len(pred_sents) > sample:
                break

        return pred_sents, actual_sents, inp_sents, prob

    def precision(self, sample=None):

        pred_sents, actual_sents, _, _ = self.predict(sample=sample)
        cer = char_error_rate(preds=pred_sents, target=actual_sents)
        wer = word_error_rate(preds=pred_sents, target=actual_sents)
        return cer, wer
    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)

        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        self.model.load_state_dict(checkpoint['state_dict'])
        self.iter = checkpoint['iter']

        self.train_losses = checkpoint['train_losses']

    def save_checkpoint(self, filename):
        state = {'iter': self.iter, 'state_dict': self.model.state_dict(),
                 'optimizer': self.optimizer.state_dict(), 'train_losses': self.train_losses,
                 'scheduler': self.scheduler.state_dict()}

        path, _ = os.path.split(filename)
        os.makedirs(path, exist_ok=True)

        torch.save(state, filename)

    def batch_to_device(self, batch):

        src = batch['src'].to(self.device, non_blocking=True)
        tgt = batch['tgt'].to(self.device, non_blocking=True)

        batch = {
            'src': src,
            'tgt': tgt
        }

        return batch
    def load_weights(self, filename):
        state_dict = torch.load(filename, map_location=torch.device(self.device))

        for name, param in self.model.named_parameters():
            if name not in state_dict:
                print(f'{name} not found')
            elif state_dict[name].shape != param.shape:
                print(
                    f'{name} missmatching shape, required {param.shape} but found {state_dict[name].shape}'
                )
                del state_dict[name]

        self.model.load_state_dict(state_dict, strict=False)

    def save_weights(self, filename):
        path, _ = os.path.split(filename)
        os.makedirs(path, exist_ok=True)

        torch.save(self.model.state_dict(), filename)   

In [13]:
checkpoint = torch.load('/kaggle/input/checkpoint-new-data/checkpoints/checkpoint.pth')

In [14]:
model_weights = checkpoint['state_dict']

In [15]:
import re
import nltk
from collections import Counter
import torch

class Predictor:
    def __init__(self, weight_path, have_att=True):
        ENC_EMB_DIM = 256
        DEC_EMB_DIM = 256
        ENC_HID_DIM = 512
        DEC_HID_DIM = 512
        ENC_DROPOUT = 0.5
        DEC_DROPOUT = 0.5
        MAX_LEN = 46
        self.maxlen = MAX_LEN
        self.vocab = Vocab(alphabets)

        INPUT_DIM = self.vocab.__len__()
        OUTPUT_DIM = self.vocab.__len__()

        self.model = Seq2Seq(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM, encoder_embbeded=ENC_EMB_DIM,
                              decoder_embedded=DEC_EMB_DIM,
                              encoder_hidden=ENC_HID_DIM, decoder_hidden=DEC_HID_DIM, encoder_dropout=ENC_DROPOUT,
                              decoder_dropout=DEC_DROPOUT)


        self.load_weights(weight_path)
        if torch.cuda.is_available():
            self.device = "cuda"
            self.model.to('cuda')
        else:
            self.device = "cpu"

        print("Device: ", self.device)
        print("Loaded model")

    def predict_ngram(self, ngram, beamsearch=False):
        '''
          Denoise for ngram
          ngram: text
        '''
        src = self.preprocessing(ngram)
        src = src.unsqueeze(0)
        src = src.to(self.device)

        translated_sentence, prob = translate(src, self.model)
        return self.vocab.decode(translated_sentence.tolist()[0])

    def spelling_correct(self, sentence):
        # Remove characters that out of vocab
        sentence = re.sub(
            r'[^aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789!"#$%&''()*+,-./:;<=>?@[\]^_`{|}~ ]',
            "", sentence)

        # Extract pharses
        phrases, phrases_all, index_sent_dict = self.extract_phrases(sentence)

        correct_phrases = []
        for phrase in phrases:
            ngrams = list(self.gen_ngrams(phrase, n=NGRAM))
            correct_ngram_str_array = []
            for ngram_list in ngrams:
                ngram_str = " ".join(ngram_list)

                correct_ngram_str = self.predict_ngram(ngram_str)
                correct_ngram_str_array.append(correct_ngram_str)
            correct_phrase = self.reconstruct_from_ngrams(correct_ngram_str_array)
            correct_phrases.append(correct_phrase)
        return self.decode_phrases(correct_phrases, phrases_all, index_sent_dict)

    def reconstruct_from_ngrams(self, predicted_ngrams):
        '''
        predicted_ngrams: list of ngram_str
        '''

        candidates = [Counter() for _ in range(len(predicted_ngrams) + NGRAM + 46)]
        for nid, ngram in (enumerate(predicted_ngrams)):
            tokens = re.split(r' +', ngram)
            for wid, word in enumerate(tokens):
                candidates[nid + wid].update([word])
        return ' '.join(c.most_common(1)[0][0] for c in candidates if len(c) != 0)

    def extract_phrases(self, text):
        pattern = r'\w[\w ]*|\s\W+|\W+'

        phrases_all = re.findall(pattern, text)

        index_sent_dict = {}
        phrases_str = []
        for ind, phrase in enumerate(phrases_all):
            if not re.match(r'[!"#$%&''()*+,-./:;<=>?@[\]^_`{|}~]', phrase.strip()):
                phrases_str.append(phrase.strip())
                index_sent_dict[ind] = phrase

        return phrases_str, phrases_all, index_sent_dict

    def decode_phrases(self, correct_phrases, phrases, index_sent_dict):
        # correct_phrases = ['lê văn', 'Hoàng', 'Hehe', 'g']
        sentence_correct = phrases.copy()
        for i, idx_sent in enumerate(index_sent_dict.keys()):
            sentence_correct[idx_sent] = correct_phrases[i]

        # print(sentence_correct)
        return "".join(sentence_correct)

    def preprocessing(self, sentence):

        # Encode characters
        noise_sent_idxs = self.vocab.encode(sentence)

        # Padding to MAXLEN
        src_len = len(noise_sent_idxs)
        if self.maxlen - src_len < 0:
            noise_sent_idxs = noise_sent_idxs[:self.maxlen]
            src_len = len(noise_sent_idxs)
            print("Over length in src")
        src = np.concatenate((
            noise_sent_idxs,
            np.zeros(self.maxlen - src_len, dtype=np.int32)))

        return torch.LongTensor(src)

    def gen_ngrams(self, sent, n=5):
        tokens = sent.split()

        return [tokens] if len(tokens) < n else nltk.ngrams(sent.split(), n)

    def load_weights(self, filename):
        state_dict = torch.load(filename, map_location=torch.device('cpu'))

        for name, param in self.model.named_parameters():
            if name not in state_dict:
                print(f'{name} not found')
            elif state_dict[name].shape != param.shape:
                print(
                    f'{name} missmatching shape, required {param.shape} but found {state_dict[name].shape}'
                )
                del state_dict[name]

        self.model.load_state_dict(state_dict, strict=False)



In [16]:
ckp = torch.load('/kaggle/input/checkpoint-new-data/checkpoints/checkpoint.pth')

In [17]:
weight = ckp['state_dict']

In [18]:
torch.save(weight,'/kaggle/working/weight.pt')

In [19]:
predictor = Predictor(weight_path='/kaggle/working/weight.pt')

Device:  cuda
Loaded model


In [20]:
def predict_test(test_dataset):
    pred_sents = []
    actual_sents = []
    results = []
    for i in tqdm(range(len(test_dataset))):
        input_sentence = test_dataset['input'][i]
        actual_sent = test_dataset['output'][i]
        predict_sentence = predictor.spelling_correct(input_sentence)
        pred_sents.append(predict_sentence)
        actual_sents.append(actual_sent)
        results.append((predict_sentence,actual_sent))
    return pred_sents, actual_sents, results

In [21]:
input_sentence = test_dataset['input'][14]
input_sentence

'Mùa bóng này , cscCLBnước Anh đồng loạt di theo phongtrào tang giá vé vào sân , đề bù đắp chi phí khổng lồ cua nhữngphi vj chuyển nhượng và việc trả lương cầu thủ .'

In [22]:
actual_sent = test_dataset['output'][14]
actual_sent

'Mùa bóng này, các CLB nước Anh đồng loạt đi theo phong trào tăng giá vé vào sân, đề bù đắp chi phí khổng lồ của những phi vụ chuyển nhượng và việc trả lương cầu thủ.'

In [23]:
predict_sentence = predictor.spelling_correct(input_sentence)

In [24]:
predict_sentence

'Mùa bóng này, các CLB đồng loạt đi theo phong trào tăng giá vé vào, đề bù đắp chi phí khổng lồ của những phi vị chuyển và việc trả lương cầu thủ.'

In [25]:
pred_sents, actual_sents, results1 = predict_test(test_dataset)

  1%|          | 298/35999 [03:06<6:50:19,  1.45it/s]

Over length in src


 15%|█▌        | 5523/35999 [57:53<4:12:34,  2.01it/s]/tmp/ipykernel_24/1437351082.py:45: RuntimeWarning: invalid value encountered in divide
  char_probs = np.sum(char_probs, axis=-1) / (char_probs > 0).sum(-1)
 17%|█▋        | 6081/35999 [1:03:31<4:06:45,  2.02it/s]

Over length in src


 18%|█▊        | 6498/35999 [1:07:47<5:29:53,  1.49it/s]

...

 93%|█████████▎| 33546/35999 [5:46:50<18:49,  2.17it/s]

Over length in src
Over length in src


 97%|█████████▋| 34903/35999 [6:01:07<15:02,  1.21it/s]

Over length in src


100%|██████████| 35999/35999 [6:12:15<00:00,  1.61it/s]


In [26]:
df = pd.DataFrame(results1, columns=['predict_sentence', 'actual_sent'])

In [27]:
df.to_csv('output_new_data.csv', index=False)